In [37]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
import os
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import random
import h5py
import math
import pickle
from datetime import datetime
from numba import cuda 
from tensorflow.keras import Model
%matplotlib inline

In [2]:
import sys  
sys.path.insert(0, '..')

In [3]:
from tiling.read_tiles import TissueDataset, load_color_normalization_values
from tiling.preprocessing.processing import split_whole_slide
from tiling.preprocessing.datamodel import SlideManager, Slide

In [4]:
# Code snippet nvidia gpu
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [5]:
HDF5_FOLDER = '/media/nico/data/fourthbrain/project/training_CAMELYON16'
green_layer_only = False
color_normalization_file="/home/nico/Documents/FourthBrainBreastCancer/tiling/CAMELYON16_color_normalization.json"

In [6]:
# FEATURE MAP CONSTRUCTION

In [7]:
# FIRST GET TILES

In [8]:
CAM_BASE_DIR = '/media/nico/data/fourthbrain/project/'
CAM16_DIR = CAM_BASE_DIR + 'CAMELYON16/'
color_normalization_file="/home/nico/Documents/FourthBrainBreastCancer/tiling/CAMELYON16_color_normalization.json"

In [9]:
mgr = SlideManager(cam16_dir=CAM16_DIR)

In [10]:
i = 0
level = 3
tile_size=256
verbose = True

In [11]:
model = keras.models.load_model('/media/nico/data/fourthbrain/project/model_testing/model_inceptionv3.hdf5')

In [12]:
# ALL TILES HAVE DIFFERENT DIMENSIONS - WE NEED TO FEED A FEATURE MATRIX OF THE SAME SIZE IN THE UNET
# SO WE ARE GOING TO LOOK FIND THE WIDEST AND HIGHEST SLIDES IN OUR DATASETS FIRST
# FROM THE PAPER : "however the sizes of WSIs differ. Therefore we first prepare a white map of sufficient size filled with zeroes"

In [13]:
def features_2dim(mgr):
    w, h = 0, 0
    for sl in mgr.test_slides:
        (wt, ht) = sl.level_dimensions[level]
        w = wt if wt> w else w
        h = ht if ht> w else h

    for sl in mgr.annotated_slides:
        (wt, ht) = sl.level_dimensions[level]
        w = wt if wt> w else w
        h = ht if ht> w else h

    for sl in mgr.negative_slides:
        (wt, ht) = sl.level_dimensions[level]
        w = wt if wt> w else w
        h = ht if ht> w else h

    wf = int(w / tile_size)
    wf += 1 if w % tile_size != 0 else 0
    
    hf = int(h / tile_size)
    hf += 1 if h % tile_size != 0 else 0
    
    return (w, h),(wf, hf)

In [14]:
level_3_max_size, level_3_features_eq = features_2dim(mgr)

In [15]:
level_3_max_size, level_3_features_eq

((27648, 27712), (108, 109))

In [16]:
dim_map = max(level_3_features_eq)

In [17]:
#### ALRIGHT THE FEATURES MAP WILL BE max(wf, hf) so for a 256 tile 109 x 109 

In [55]:
# Model layer
layer_output = model.layers[-2].output
feature_model = Model(inputs=model.input, outputs=layer_output)

In [31]:
slide = mgr.test_slides[i]

In [63]:
slide.level_dimensions[3]

(10752, 11200)

In [56]:
iter_whole_slide = split_whole_slide(slide, level=level, tile_size=tile_size, color_normalization_file=color_normalization_file, verbose=False) 

In [57]:
tiles, labels, coords, slide_locations = list(), list(), list(), list()
feature_map = list()

y_preds, layer_outputs = list(), list()
for tile, label, coord, slide_location in iter_whole_slide:
    tiles.append(tile)
    labels.append(label)
    coords.append(coord)
    slide_locations.append(slide_location)
    feature_map.append(feature_model.predict(tiles[0].reshape(1,256,256,3)))

In [73]:
feature_map[0].shape[1]

2048

In [74]:
d = np.zeros((dim_map, dim_map, feature_map[0].shape[1]))

In [77]:
for i in range(0, len(coords)):
    d[coords[i][0], coords[i][1],:] = feature_map[i]